In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb
import random

In [2]:
data = pd.read_csv("minute_data/BTC-USD_1M_SIGNALS.csv",)

In [3]:
data

,Unix,Date,Symbol,Open,High,Low,Close,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1514764620,2017-12-31 23:57:00,BTC/USD,13908.73,13913.26,13874.99,13913.26,-0.295570,-0.514960,2.660454,0.163399,0.363161,0.229565,-0.163178,3.398648e-14
1,1514764560,2017-12-31 23:56:00,BTC/USD,13827.00,13908.69,13827.00,13859.58,-0.303185,-0.684081,0.796397,0.087233,0.089257,2.307135,-0.163178,3.398648e-14
2,1514764500,2017-12-31 23:55:00,BTC/USD,13825.05,13825.05,13825.05,13825.05,-0.344218,-0.442047,-0.293868,-0.036330,-0.255116,-0.190940,-0.163178,3.398648e-14
3,1514764440,2017-12-31 23:54:00,BTC/USD,13884.14,13884.14,13823.88,13854.28,-0.020416,0.373551,-0.691035,-0.079503,-0.318178,-0.296745,1.710867,3.398648e-14
4,1514764380,2017-12-31 23:53:00,BTC/USD,13854.52,13898.70,13840.85,13884.15,0.367259,0.380942,-0.266264,-0.058034,-0.197810,-0.296745,0.298594,3.398648e-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675129,1609459500,2021-01-01 00:05:00,BTC/USD,29021.86,29023.38,28982.33,28999.50,-0.196942,0.012214,0.659112,0.156727,1.550490,0.483496,-0.163178,-3.784359e-01
2675130,1609459440,2021-01-01 00:04:00,BTC/USD,29048.13,29057.73,29035.61,29045.19,-0.141744,0.278150,1.064047,0.340786,1.726775,-0.296745,0.926236,-4.489000e-01
2675131,1609459380,2021-01-01 00:03:00,BTC/USD,29037.68,29069.39,29019.00,29048.13,-0.035411,0.017070,1.176341,0.486452,1.766349,0.604494,-0.163178,-5.712855e-01
2675132,1609459320,2021-01-01 00:02:00,BTC/USD,29069.80,29073.02,29028.14,29035.89,0.084682,-0.075473,1.032994,0.572733,1.641080,-0.296745,-0.048052,-6.535419e-01


In [4]:
predict_data = data.copy().drop(["Open","Close","High","Low", "Symbol"],axis=1)
max_days = 5
target_range = 2
for i in range(1,max_days):#2jours
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i), "ADX-{}".format(i), "+DM-{}".format(i), "-DM-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG","ADX14","+DM","-DM"]].shift(i)
    predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","Volume USD","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

In [5]:
predict_data = predict_data[[i % int(max_days) == 0 for i in range(len(predict_data))]]

In [6]:
predict_data

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,...,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Target
0,1514764380,2017-12-31 23:53:00,0.367259,0.380942,-0.266264,-0.058034,-0.197810,-0.296745,0.298594,3.398648e-14,...,-0.303185,0.796397,0.087233,0.089257,-0.514960,-0.295570,2.660454,0.163399,0.363161,1
2,1514764260,2017-12-31 23:51:00,-0.307925,0.022998,0.058271,-0.026308,-0.051013,-0.296745,0.753071,3.398648e-14,...,-0.020416,-0.691035,-0.079503,-0.318178,-0.442047,-0.344218,-0.293868,-0.036330,-0.255116,1
4,1514764140,2017-12-31 23:49:00,-0.131674,-0.000092,0.432820,0.133421,0.341924,-0.296745,0.311280,3.398648e-14,...,0.140478,0.068701,-0.042420,-0.116987,0.380942,0.367259,-0.266264,-0.058034,-0.197810,1
6,1514764020,2017-12-31 23:47:00,-0.348619,-0.010254,0.536837,0.273205,0.545667,-0.296745,-0.163178,3.398648e-14,...,-0.309888,0.076674,0.063604,0.196784,0.022998,-0.307925,0.058271,-0.026308,-0.051013,0
8,1514763900,2017-12-31 23:45:00,-0.322627,0.521257,0.343099,0.251670,0.320053,-0.242486,-0.163178,3.398648e-14,...,0.067075,0.433254,0.213927,0.486277,-0.000092,-0.131674,0.432820,0.133421,0.341924,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675118,1609459920,2021-01-01 00:12:00,-0.123238,0.541444,-0.097895,-0.822243,0.166855,-0.296745,0.905304,3.879120e-01,...,-0.144609,-0.599621,-0.852356,0.044756,0.028691,-0.183870,-0.424827,-0.774573,0.312876,1
2675120,1609459800,2021-01-01 00:10:00,-0.139513,0.224667,0.126447,-0.591698,0.670025,-0.296745,0.116549,4.157917e-01,...,1.640388,-0.715009,-0.946685,-0.202488,0.056217,1.563090,-0.454848,-0.886675,-0.054885,1
2675122,1609459680,2021-01-01 00:08:00,-0.025056,0.022950,0.340013,-0.326229,1.055277,-0.296745,0.016964,5.271002e-02,...,1.059105,-0.233153,-0.735235,0.363404,0.541444,-0.123238,-0.097895,-0.822243,0.166855,1
2675124,1609459560,2021-01-01 00:06:00,-0.082574,0.041965,1.062017,0.025713,1.505349,-0.296745,0.265610,-2.630277e-01,...,-0.010583,0.128702,-0.449689,0.911287,0.224667,-0.139513,0.126447,-0.591698,0.670025,1


In [7]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [8]:
from sklearn.model_selection import train_test_split

In [12]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date", "Unix", "Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date", "Unix", "Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [14]:
algo = "MLP"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(200,2000,200),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=False)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

In [50]:
model.score(test_data,test_labels)

0.5018050541516246

In [51]:
model.score(train_data,train_labels)

1.0

In [52]:
from sklearn.metrics import confusion_matrix

In [53]:
preds = model.predict(test_data)

In [54]:
conf = confusion_matrix(test_labels,preds)
conf

array([[53, 72],
       [66, 86]], dtype=int64)

In [55]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.44537815126050423

In [56]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.5443037974683544

## Cross Validation

In [19]:
def cross_validation_loop(predict_data, model_name):
    scores = []
    TPs = []
    TNs = []
    for randomize in range(1,20):
        train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=randomize)
        if model_name == "MLP":
            model = MLPClassifier(hidden_layer_sizes=(200,2000,200))
        elif model_name == "CAT":
            model = CatBoostClassifier(iterations=100,depth=12,verbose=False)
        elif model_name == "RF":
            model = RandomForestClassifier(n_estimators=100,verbose=False)
        model.fit(train_data, train_labels)
        scores.append(model.score(test_data, test_labels))
        conf = confusion_matrix(test_labels,model.predict(test_data))
        TN = conf[0,0] / (conf[0,0] + conf[1,0])
        TP = conf[1,1] / (conf[1,1] + conf[0,1])
        TPs.append(TP)
        TNs.append(TN)
        print("Score : {} || TP : {} || TN : {}".format(scores[-1],TP,TN))


    return scores,TPs,TNs

In [20]:
#scores,TPs,TNs = cross_validation_loop(predict_data,"CAT")

In [24]:
from statistics import mean

In [50]:
print("CatBoost score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5067452023560707 || TP moyen : 0.5411620348700146 || TN moyen : 0.4619500842660272


In [22]:
scores,TPs,TNs = cross_validation_loop(predict_data,"RF")

Score : 0.51985559566787 || TP : 0.5694444444444444 || TN : 0.46616541353383456
Score : 0.49458483754512633 || TP : 0.5194805194805194 || TN : 0.4634146341463415
Score : 0.5054151624548736 || TP : 0.5465116279069767 || TN : 0.4380952380952381
Score : 0.48375451263537905 || TP : 0.5136986301369864 || TN : 0.45038167938931295
Score : 0.4693140794223827 || TP : 0.4722222222222222 || TN : 0.4639175257731959
Score : 0.48736462093862815 || TP : 0.525 || TN : 0.4358974358974359
Score : 0.51985559566787 || TP : 0.5280898876404494 || TN : 0.5050505050505051
Score : 0.49458483754512633 || TP : 0.5611510791366906 || TN : 0.427536231884058
Score : 0.49458483754512633 || TP : 0.5384615384615384 || TN : 0.4380165289256198
Score : 0.4657039711191336 || TP : 0.4899328859060403 || TN : 0.4375
Score : 0.4693140794223827 || TP : 0.5205479452054794 || TN : 0.4122137404580153
Score : 0.5306859205776173 || TP : 0.5714285714285714 || TN : 0.46788990825688076
Score : 0.4584837545126354 || TP : 0.5 || TN : 0.4

In [25]:
print("Random Forest score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

Random Forest score moyen : 0.5040851225536767 || TP moyen : 0.538149033546309 || TN moyen : 0.4600308310035195


In [26]:
scores,TPs,TNs = cross_validation_loop(predict_data,"MLP")

Score : 0.51985559566787 || TP : 0.5675675675675675 || TN : 0.46511627906976744
Score : 0.48014440433212996 || TP : 0.5068493150684932 || TN : 0.45038167938931295
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5234657039711191 || TP : 0.5496688741721855 || TN : 0.49206349206349204
Score : 0.4657039711191336 || TP : 0.46308724832214765 || TN : 0.46875
Score : 0.5342960288808665 || TP : 0.5734265734265734 || TN : 0.4925373134328358
Score : 0.5270758122743683 || TP : 0.541095890410959 || TN : 0.5114503816793893
Score : 0.5054151624548736 || TP : 0.5657894736842105 || TN : 0.432
Score : 0.48014440433212996 || TP : 0.5266666666666666 || TN : 0.4251968503937008
Score : 0.5090252707581228 || TP : 0.5302013422818792 || TN : 0.484375
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5451263537906137 || TP : 0.5933333333333334 || TN : 0.4881889763779528
Score : 0.4729241877256318 || TP : 0.5131578947368421 || TN : 0.424
Score : 0.534296028

In [27]:
print("MLP score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

MLP score moyen : 0.5078852365570967 || TP moyen : 0.5443692036536304 || TN moyen : 0.4653102613033886


### Using TensorFlow

In [21]:
%load_ext tensorboard

In [22]:
!rm -rf ./logs/

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [72]:
shape = (len(predict_data.columns)-2,)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [73]:
print(train_data.shape, test_data.shape)

(562, 48) (277, 48)


In [74]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [75]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [76]:
model.fit(train_data, train_labels, epochs=30, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback])

Epoch 1/30
18/18 [==============================] - 4s 142ms/step - loss: 0.7538 - accuracy: 0.5320 - precision: 0.5391 - recall: 0.8087 - val_loss: 0.6873 - val_accuracy: 0.5487 - val_precision: 0.5487 - val_recall: 1.0000
Epoch 2/30
18/18 [==============================] - 2s 102ms/step - loss: 0.6805 - accuracy: 0.5302 - precision: 0.5304 - recall: 0.9966 - val_loss: 0.6878 - val_accuracy: 0.5668 - val_precision: 0.5769 - val_recall: 0.7895
Epoch 3/30
18/18 [==============================] - 2s 103ms/step - loss: 0.6691 - accuracy: 0.5765 - precision: 0.5743 - recall: 0.7785 - val_loss: 0.6927 - val_accuracy: 0.5307 - val_precision: 0.5775 - val_recall: 0.5395
Epoch 4/30
18/18 [==============================] - 2s 111ms/step - loss: 0.6651 - accuracy: 0.6139 - precision: 0.6421 - recall: 0.6141 - val_loss: 0.7008 - val_accuracy: 0.5523 - val_precision: 0.5854 - val_recall: 0.6316
Epoch 5/30
18/18 [==============================] - 2s 104ms/step - loss: 0.6455 - accuracy: 0.6423 - pr

In [28]:
test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

4/4 - 0s - loss: 3.1906 - accuracy: 0.4414 - precision: 0.5116 - recall: 0.3492 - 28ms/epoch - 7ms/step


ValueError: too many values to unpack (expected 2)

In [147]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])



In [148]:
preds = np.argmax(probability_model.predict(test_data),axis=1)

In [149]:
conf = confusion_matrix(test_labels,preds)
conf

array([[51, 74],
       [62, 90]], dtype=int64)

In [150]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.45132743362831856

In [151]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.5487804878048781